# Deliberative Agent Chatbot

In [ ]:
%pip install -q gradio

In [ ]:
import logging

logging.basicConfig(
    #filename='logikon_chatbot.log',
    #filemode='w',
    format='%(asctime)s %(levelname)-8s %(message)s',
    level=logging.DEBUG,
    datefmt='%Y-%m-%d %H:%M:%S'
)

In [ ]:
import logikon

logging.info(f"Installed `logikon` module version: {logikon.__version__}")

In [ ]:
#from langchain_openai import ChatOpenAI
from logikon.backends.chat_models_with_grammar import create_logits_model

inference_server_url = "http://localhost:8000/v1"
model_id = "openchat/openchat_3.5"

llm = create_logits_model(
    model_id=model_id,
    inference_server_url=inference_server_url,
    api_key="EMPTY",
    llm_backend="VLLM",
    max_tokens=780,
    temperature=0.4,
)

In [ ]:
# TEST SERVER
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("tell me a short joke about {topic}")
output_parser = StrOutputParser()
chain = prompt | llm | output_parser
chain.invoke("farmers")

In [ ]:
import re
from typing import Tuple

import gradio as gr
from langchain_core.messages import AIMessage, HumanMessage

from logikon.guides.proscons.recursive_balancing_guide import RecursiveBalancingGuide, RecursiveBalancingGuideConfig

guide_config = RecursiveBalancingGuideConfig(
    expert_model = "openchat/openchat_3.5",
    inference_server_url = "http://localhost:8000/v1",
    api_key = "EMPTY"
)
guide = RecursiveBalancingGuide(tourist_llm=llm, config=guide_config)

EXAMPLES = [
    ("We're a nature-loving family with three kids, have some money left, and no plans "
     "for next week-end. Should we visit Disneyland?"),
    "Should I stop eating animals?",
    "Bob needs a reliable and cheap car. Should he buy a Mercedes?",
    ('Gavin has an insurance policy that includes coverage for "General Damages," '
     'which includes losses from "missed employment due to injuries that occur '
     'under regular working conditions."\n\n'
     'Gavin works as an A/C repair technician in a small town. One day, Gavin is '
     'hired to repair an air conditioner located on the second story of a building. '
     'Because Gavin is an experienced repairman, he knows that the safest way to '
     'access the unit is with a sturdy ladder. While climbing the ladder, Gavin '
     'loses his balance and falls, causing significant injury. Because of this, he '
     'subsequently has to stop working for weeks. Gavin files a claim with his '
     'insurance company for lost income.\n\n'
     'Does Gavin\'s insurance policy cover his claim for lost income?'),
     "How many arguments did you consider in your internal reasoning? (Brief answer, please.)",
     "Did you consider any counterarguments in your internal reasoning?",
     "From all the arguments you considered and assessed, which one is the most important?",
     "Did you refute any arguments or reasons for lack of plausibility?"
]


def add_details(response: str, reasoning: str, svg_argmap: str) -> str:
    """Add reasoning details to the response message shown in chat."""
    response_with_details = (
        f"<p>{response}</p>"
        '<details id="reasoning">'
        "<summary><i>Internal reasoning trace</i></summary>"
        f"<code>{reasoning}</code></details>"
        '<details id="svg_argmap">'
        "<summary><i>Argument map</i></summary>"
        f"\n<div>\n{svg_argmap}\n</div>\n</details>"
    )
    return response_with_details


def get_details(response_with_details: str) -> Tuple[str, dict[str, str]]:
    """Extract response and details from response_with_details shown in chat."""
    if "<details id=" not in response_with_details:
        return response_with_details, {}
    details_dict = {}
    response, *details_raw = response_with_details.split('<details id="')
    for details in details_raw:
        details_id, details_content = details.split('"', maxsplit=1)
        details_content = details_content.strip()
        if details_content.endswith("</code></details>"):
            details_content = details_content.split("<code>")[1].strip()
            details_content = details_content[:-len("</code></details>")].strip()
        elif details_content.endswith("</div></details>"):
            details_content = details_content.split("<div>")[1].strip()
            details_content = details_content[:-len("</div></details>")].strip()
        else:
            logging.warning(f"Unrecognized details content: {details_content}")
            details_content = "UNRECOGNIZED DETAILS CONTENT"
        details_dict[details_id] = details_content
    return response, details_dict


def remove_links_svg(svg):
    svg = svg.replace("</a>","")
    svg = svg.replace("\n\n","\n")
    regex = r"<a xlink[^>]*>"
    svg = re.sub(regex, "", svg, count=0, flags=re.MULTILINE)
    return svg


def resize_svg(svg, max_width=800):
    regex = r"<svg width=\"(?P<width>[\d]+)pt\" height=\"(?P<height>[\d]+)pt\""
    match = next(re.finditer(regex, svg, re.MULTILINE))
    width = int(match.group("width"))
    height = int(match.group("height"))
    if width <= max_width:
        return svg

    scale = max_width / width
    s_width = round(scale * width)
    s_height = round(scale * height)
    s_svg = svg.replace(match.group(), f'<svg width="{s_width}pt" height="{s_height}pt"')
    return s_svg

def postprocess_svg(svg):
    svg = "<svg" + svg.split("<svg", maxsplit=1)[1]
    svg = remove_links_svg(svg)
    svg = resize_svg(svg, max_width=800)
    return svg

async def predict(message, history):
    """Predict the response for the given message.
    Args:
    message: dict, the message to predict the response for.
    history: list, the session-state history as shown to user in the chat interface.

    Returns:
    str, the predicted response for the given message
    """
    history_langchain_format = []  # History in LangChain format, as shown to the LLM
    for human, ai in history:
        history_langchain_format.append(HumanMessage(content=human))
        response, details = get_details(ai)
        logging.debug(f"Details: {details}")
        history_langchain_format.append(AIMessage(content=response))
        if "reasoning" in details:
            content = f"Internal reasoning trace (hidden from user): {details['reasoning']}"
            history_langchain_format.append(AIMessage(content=content))
    logging.debug(f"Message: {message}")
    logging.debug(f"History: {history}")
    history_langchain_format.append(HumanMessage(content=message["text"]))

    if len(history_langchain_format) <= 1:
        # use guide always and exclusively at first turn
        response, artifacts = await guide(message["text"])
        svg = postprocess_svg(artifacts["svg"])
        response = add_details(response, artifacts["protocol"], svg)

    else:
        response = llm.invoke(history_langchain_format).content

    return response


bot = gr.ChatInterface(
    predict,
    title="Logikon's Deliberative Agent Bot (Guided Reasoning Demo)",
    multimodal=True,
    examples=[{"text": e, "files":[]} for e in EXAMPLES]
)
bot.launch(auth=("x123","123"), share=True)